## Goods

In [1]:
import numpy as np 
import pandas as pd

goods = pd.read_csv('goods.csv', sep=';')

print('Size of the data set: {} rows and {} columns'.format(*goods.shape))
goods.head()

Size of the data set: 34316 rows and 5 columns


,id,name,price,oldPrice,visible
0,617,Носки детские,69.0,69.0,0.0
1,618,Рейтузы детские для девочек,349.0,449.0,0.0
2,686,Трусы детские для девочек 3шт,249.0,249.0,0.0
3,687,Трусы детские для девочек 3шт,249.0,249.0,0.0
4,688,Трусы детские для девочек 2шт,199.0,299.0,0.0


In [2]:
for col in goods.columns:
    print('{} missing values in {} column'.format(goods[col].isnull().sum().sum(), col))

0 missing values in id column
1 missing values in name column
1 missing values in price column
1 missing values in oldPrice column
1 missing values in visible column


In [3]:
goods.dropna(inplace=True)
print('Size of the data set: {} rows and {} columns'.format(*goods.shape))

Size of the data set: 34315 rows and 5 columns


## Order goods

In [4]:
order_goods = pd.read_csv('order-goods.csv', sep=';', 
                          encoding='ISO-8859-1', dtype={'price': 'object'})

order_goods.price = order_goods.price.apply(lambda x: float((str(x).replace(' ', ''))))

print('Size of the data set: {} rows and {} columns'.format(*order_goods.shape))
order_goods.head()

Size of the data set: 897933 rows and 7 columns


,orderId,goodId,size,qty,price,originalPrice,originalOldPrice
0,451752.0,37101.0,70B,2.0,1.0,499.0,1099.0
1,451752.0,37129.0,S,2.0,499.0,499.0,0.0
2,451752.0,42809.0,one size,1.0,1.0,399.0,0.0
3,451752.0,37101.0,75C,1.0,499.0,499.0,1099.0
4,451752.0,42784.0,S,1.0,1.0,499.0,0.0


In [5]:
for col in order_goods.columns:
    print('{} missing values in {} column'.format(order_goods[col].isnull().sum().sum(), col))

3889 missing values in orderId column
1 missing values in goodId column
1 missing values in size column
1 missing values in qty column
1 missing values in price column
1 missing values in originalPrice column
1 missing values in originalOldPrice column


In [6]:
# order_goods[pd.isnull(order_goods.orderId) == True].head(2)
order_goods.dropna(inplace=True)

print('Size of the data set: {} rows and {} columns'.format(*order_goods.shape))

Size of the data set: 894044 rows and 7 columns


In [7]:
order_goods.groupby(order_goods.columns.tolist(), as_index=False).size().head()

orderId   goodId   size  qty  price   originalPrice  originalOldPrice
173042.0  19546.0  146   1.0  599.0   299.0          599.0               1
          19549.0  140   1.0  599.0   299.0          599.0               1
          23192.0  140   1.0  599.0   599.0          0.0                 1
          23206.0  140   1.0  1499.0  599.0          1499.0              1
          23223.0  146   1.0  699.0   499.0          699.0               1
dtype: int64

## Orders

In [8]:
orders = pd.read_csv('orders.csv', sep=';', dtype={'erp_status': 'object'})

orders.accept_time = pd.to_datetime(orders.accept_time)

print('Size of the data set: {} rows and {} columns'.format(*orders.shape))
orders.head()

Size of the data set: 174063 rows and 9 columns


,id,from_site,status,payment_type,payed,accept_time,total_cost,user_id,erp_status
0,451752.0,inflin,6.0,cash,0.0,2017-05-31 23:45:40,2399.0,357892.0,-
1,451751.0,inflin,6.0,cash,0.0,2017-05-31 23:44:14,2200.0,82507.0,-
2,451750.0,concept,6.0,cash,0.0,2017-05-31 23:39:14,6000.0,316491.0,-
3,451749.0,acoola,6.0,cash,0.0,2017-05-31 23:37:19,2101.0,336144.0,-
4,451746.0,concept,6.0,cash,0.0,2017-05-31 23:27:22,6300.0,332755.0,-


In [9]:
for col in orders.columns:
    print('{} missing values in {} column'.format(orders[col].isnull().sum().sum(), col))

1371 missing values in id column
1 missing values in from_site column
1 missing values in status column
74 missing values in payment_type column
1 missing values in payed column
1 missing values in accept_time column
1 missing values in total_cost column
1 missing values in user_id column
1 missing values in erp_status column


In [10]:
orders.dropna(inplace=True)
print('Size of the data set: {} rows and {} columns'.format(*orders.shape))

Size of the data set: 172619 rows and 9 columns


In [11]:
# Статусы заказов: 
# 3 - Оплачен, формируется к отправке
# 5 - Отменен
# 6 - Отправлен 
# 7 - Продажа оформлена

orders.status.value_counts()

6.0    148032
5.0     24540
3.0        43
7.0         4
Name: status, dtype: int64

## Data

In [12]:
orders[orders.status == 6].shape

(148032, 9)

In [13]:
order_goods.rename(columns={'orderId': 'order_id'}, inplace=True)
orders.rename(columns={'id': 'order_id'}, inplace=True)

In [14]:
data = pd.merge(orders[orders.status == 6], order_goods, how='inner', on='order_id')
data.shape

(585924, 15)

In [15]:
# orders[orders.user_id == 82507.0]
# order_goods[order_goods.order_id == 451751.0]
# order_goods.price[order_goods.order_id == 451751.0].sum()

data.head(2)

,order_id,from_site,status,payment_type,payed,accept_time,total_cost,user_id,erp_status,goodId,size,qty,price,originalPrice,originalOldPrice
0,451752.0,inflin,6.0,cash,0.0,2017-05-31 23:45:40,2399.0,357892.0,-,37101.0,70B,2.0,1.0,499.0,1099.0
1,451752.0,inflin,6.0,cash,0.0,2017-05-31 23:45:40,2399.0,357892.0,-,37129.0,S,2.0,499.0,499.0,0.0


## RFM table

In [29]:
import datetime as dt

now = dt.datetime(2017, 5, 31)

In [30]:
rfm_table = data.groupby('user_id').agg({'accept_time': lambda x: (now - x.max()).days, # Recency
                                         'order_id': lambda x: len(x), # Frequency
                                         'price': lambda x: x.sum()}) # Monetary Value

rfm_table.rename(columns={'accept_time': 'recency', 
                          'order_id': 'frequency', 
                          'price': 'monetary_value'}, inplace=True)
rfm_table.head()

,monetary_value,frequency,recency
user_id,,,
48.0,21541.0,19.0,76
70.0,6308.0,7.0,307
102.0,8804.0,6.0,446
144.0,3912.0,3.0,423
151.0,1699.0,3.0,157


In [31]:
data.price[data.user_id == 70.0].sum()

6308.0

In [32]:
data.order_id[data.user_id == 70.0].count()

7

In [33]:
(now - dt.datetime(2016, 7, 27)).days

308

## RFM Quartiles

In [34]:
quantiles = rfm_table.quantile(q=[0.25,0.5,0.75])
quantiles

,monetary_value,frequency,recency
0.25,2000.0,2.0,90.0
0.50,3600.0,5.0,228.0
0.75,7893.0,10.0,483.0


In [35]:
quantiles = quantiles.to_dict()
quantiles

{'frequency': {0.25: 2.0, 0.5: 5.0, 0.75: 10.0},
 'monetary_value': {0.25: 2000.0, 0.5: 3600.0, 0.75: 7893.0},
 'recency': {0.25: 90.0, 0.5: 228.0, 0.75: 483.0}}

## RFM segmentation table

In [36]:
rfm_segmentation = rfm_table

In [37]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x, p, d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x, p, d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [38]:
rfm_segmentation['R_Quartile'] = rfm_segmentation['recency'].apply(RClass, args=('recency', quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['frequency'].apply(FMClass, args=('frequency', quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['monetary_value'].apply(FMClass, args=('monetary_value', quantiles,))

In [40]:
rfm_segmentation['RFMClass'] = rfm_segmentation.R_Quartile.map(str) \
                             + rfm_segmentation.F_Quartile.map(str) \
                             + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,monetary_value,frequency,recency,R_Quartile,F_Quartile,M_Quartile,RFMClass
user_id,,,,,,,
48.0,21541.0,19.0,76,1,1,1,111
70.0,6308.0,7.0,307,3,2,2,322
102.0,8804.0,6.0,446,3,2,1,321
144.0,3912.0,3.0,423,3,3,2,332
151.0,1699.0,3.0,157,2,3,4,234


## Top 5 best customers

In [42]:
rfm_segmentation[rfm_segmentation['RFMClass']=='111'].sort_values('monetary_value', ascending=False).head(5)

,monetary_value,frequency,recency,R_Quartile,F_Quartile,M_Quartile,RFMClass
user_id,,,,,,,
294075.0,923006.0,873.0,18,1,1,1,111
62795.0,514266.0,622.0,33,1,1,1,111
224807.0,508367.0,856.0,2,1,1,1,111
92996.0,439436.0,581.0,13,1,1,1,111
296914.0,397398.0,822.0,9,1,1,1,111
